## 나주시 농업기상정보시스템
* 주소 : https://weather.naju.go.kr/

In [8]:
import pandas as pd
from numpy import nan
import time

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup

### bs로 크롤링하기

# 날짜, 온도 빈값 처리 확인 테스트
* 2013년 1월
* 시간 없는 날짜
    * 1/4 18시

In [20]:
# webdriver로 페이지 접속
driver = webdriver.Chrome("chromedriver_94")
url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
driver.get(url)
time.sleep(1)

all_date_list = []
all_tp_list = []
for m in [1]:
    for d in range(1,32):
        # 관측기간 xpath : //*[@id="timed_date"]
        # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
        search = driver.find_element_by_xpath('//*[@id="timed_date"]')
        button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
        # 입력창 클릭 후 내용 지우고 입력하기
        search.click()
        search.clear()
        search.send_keys(f"2013{m:0>2}{d:0>2}")
        button.click()
        time.sleep(0.5)
        # bs에 담기
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        
        # 시간, 기온 기록
        td = soup.find_all('td')
        
        # 시간 기록
        # 날짜만 가져오기
        num = int(len(td)/13)
        date_list = []
        for i in range(num):
            if len(td[i*13].text) == 16:
                date_list.append(td[i*13].text)
                date_list = set(date_list)
                date_list = list(date_list)
        # 날짜 순서 정렬 및 24시날짜 하루 당기기
        # 각 달 마지막 날짜 처리
        date_list.sort()
        e = d+1
        n = m+1
        if f"2013-{m:0>2}-{e:0>2} 00:00" in date_list:
            a = date_list.index(f"2013-{m:0>2}-{e:0>2} 00:00")
            date_list[a] = f"2013-{m:0>2}-{d:0>2} 24:00"
        if f"2013-{n:0>2}-01 00:00" in date_list:
            a = date_list.index(f"2013-{n:0>2}-01 00:00")
            date_list[a] = f"2013-{m:0>2}-{d:0>2} 24:00"
        length = len(date_list)
        
        # 시간 샘플이 비어있는 부분 확인하기
        # 시간 샘플
        sample_date = []
        for i in range(1,25):
            tmp = f'2013-{m:0>2}-{d:0>2} {i:0>2}:00'
            sample_date.append(tmp)
            
        # 비어 있는 부분 확인
        if length < 24:
            empty_index_list = []
            for i, one in enumerate(sample_date):
                if one not in date_list:
                    empty_index_list.append(i)
            
            # 온도 기록
            # 날짜 만큼만 온도 가져오기
            tp_list = []
            for i in range(length):
                j = i * 13 + 1
                if td[j] == nan:
                    tp_list.append(nan)
                else:
                    tp_list.append(td[j].text)
            # 날짜에 맞춰 온도 순서 뒤집기
            new_tp_list = []
            for i in range(length-1,-1,-1):
                tmp = tp_list[i]
                new_tp_list.append(tmp)
            
            # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
            for i in empty_index_list:
                date_list.insert(i, sample_date[i])
                new_tp_list.insert(i, nan)
            
        if length == 24:
            # 온도 기록
            # 날짜 만큼만 온도 가져오기
            tp_list = []
            for i in range(length):
                j = i * 13 + 1
                if td[j] == nan:
                    tp_list.append(nan)
                else:
                    tp_list.append(td[j].text)
            # 날짜에 맞춰 온도 순서 뒤집기
            new_tp_list = []
            for i in range(length-1,-1,-1):
                tmp = tp_list[i]
                new_tp_list.append(tmp)
        
        # 날짜, 온도 리스트에 추가
        all_date_list.extend(date_list)
        all_tp_list.extend(new_tp_list)

In [21]:
print(len(all_date_list), len(all_tp_list))

744 744


In [22]:
# 데이터 셋 만들기
df = pd.DataFrame({"연-월-일 시간":all_date_list, "기온":all_tp_list})
df.loc[df['기온']=='-', '기온'] = nan

In [23]:
df2 = pd.DataFrame({"연-월-일 시간":all_date_list, "기온":all_tp_list})

In [24]:
pd.options.display.max_rows = 1000

In [25]:
df

,연-월-일 시간,기온
0,2013-01-01 01:00,-6.4
1,2013-01-01 02:00,-6.0
2,2013-01-01 03:00,-5.7
3,2013-01-01 04:00,-5.8
4,2013-01-01 05:00,-6.3
5,2013-01-01 06:00,-6.0
6,2013-01-01 07:00,-5.8
7,2013-01-01 08:00,-6.3
8,2013-01-01 09:00,-4.5
9,2013-01-01 10:00,-3.2


In [26]:
df2

,연-월-일 시간,기온
0,2013-01-01 01:00,-6.4
1,2013-01-01 02:00,-6.0
2,2013-01-01 03:00,-5.7
3,2013-01-01 04:00,-5.8
4,2013-01-01 05:00,-6.3
5,2013-01-01 06:00,-6.0
6,2013-01-01 07:00,-5.8
7,2013-01-01 08:00,-6.3
8,2013-01-01 09:00,-4.5
9,2013-01-01 10:00,-3.2


# 날짜, 온도 빈값 처리 확인 테스트
* 2013년 3월
* 온도 데이터 없는 날짜
    * 3/9 2개
    * 3/11 1개
    * 3/14 3개
    * 3/15 1개

In [12]:
# webdriver로 페이지 접속
driver = webdriver.Chrome("chromedriver_94")
url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
driver.get(url)
time.sleep(1)

all_date_list = []
all_tp_list = []
for m in [3]:
    for d in range(1,32):
        # 관측기간 xpath : //*[@id="timed_date"]
        # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
        search = driver.find_element_by_xpath('//*[@id="timed_date"]')
        button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
        # 입력창 클릭 후 내용 지우고 입력하기
        search.click()
        search.clear()
        search.send_keys(f"2013{m:0>2}{d:0>2}")
        button.click()
        time.sleep(0.5)
        # bs에 담기
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        
        # 시간, 기온 기록
        td = soup.find_all('td')
        
        # 시간 기록
        # 날짜만 가져오기
        num = int(len(td)/13)
        date_list = []
        for i in range(num):
            if len(td[i*13].text) == 16:
                date_list.append(td[i*13].text)
                date_list = set(date_list)
                date_list = list(date_list)
        # 날짜 순서 정렬 및 24시날짜 하루 당기기
        # 각 달 마지막 날짜 처리
        date_list.sort()
        e = d+1
        n = m+1
        if f"2013-{m:0>2}-{e:0>2} 00:00" in date_list:
            a = date_list.index(f"2013-{m:0>2}-{e:0>2} 00:00")
            date_list[a] = f"2013-{m:0>2}-{d:0>2} 24:00"
        if f"2013-{n:0>2}-01 00:00" in date_list:
            a = date_list.index(f"2013-{n:0>2}-01 00:00")
            date_list[a] = f"2013-{m:0>2}-{d:0>2} 24:00"
        length = len(date_list)
        
        # 시간 샘플이 비어있는 부분 확인하기
        # 시간 샘플
        sample_date = []
        for i in range(1,25):
            tmp = f'2013-{m:0>2}-{d:0>2} {i:0>2}:00'
            sample_date.append(tmp)
            
        # 비어 있는 부분 확인
        if length < 24:
            empty_index_list = []
            for i, one in enumerate(sample_date):
                if one not in date_list:
                    empty_index_list.append(i)
            
            # 온도 기록
            # 날짜 만큼만 온도 가져오기
            tp_list = []
            for i in range(length):
                j = i * 13 + 1
                if td[j] == nan:
                    tp_list.append(nan)
                else:
                    tp_list.append(td[j].text)
            # 날짜에 맞춰 온도 순서 뒤집기
            new_tp_list = []
            for i in range(length-1,-1,-1):
                tmp = tp_list[i]
                new_tp_list.append(tmp)
            
            # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
            for i in empty_index_list:
                date_list.insert(i, sample_date[i])
                new_tp_list.insert(i, nan)
            
        if length == 24:
            # 온도 기록
            # 날짜 만큼만 온도 가져오기
            tp_list = []
            for i in range(length):
                j = i * 13 + 1
                if td[j] == nan:
                    tp_list.append(nan)
                else:
                    tp_list.append(td[j].text)
            # 날짜에 맞춰 온도 순서 뒤집기
            new_tp_list = []
            for i in range(length-1,-1,-1):
                tmp = tp_list[i]
                new_tp_list.append(tmp)
        
        # 날짜, 온도 리스트에 추가
        all_date_list.extend(date_list)
        all_tp_list.extend(new_tp_list)

In [13]:
print(len(all_date_list), len(all_tp_list))

744 744


In [14]:
# 데이터 셋 만들기
df2 = pd.DataFrame({"연-월-일 시간":all_date_list, "기온":all_tp_list})

In [15]:
# 데이터 셋 만들기
df = pd.DataFrame({"연-월-일 시간":all_date_list, "기온":all_tp_list})
df.loc[df['기온']=='-', '기온'] = nan

In [17]:
pd.options.display.max_rows = 1000

In [18]:
df2

,연-월-일 시간,기온
0,2013-03-01 01:00,10.6
1,2013-03-01 02:00,10.5
2,2013-03-01 03:00,11.0
3,2013-03-01 04:00,10.3
4,2013-03-01 05:00,9.6
5,2013-03-01 06:00,9.0
6,2013-03-01 07:00,5.9
7,2013-03-01 08:00,5.2
8,2013-03-01 09:00,3.6
9,2013-03-01 10:00,3.7


In [19]:
df

,연-월-일 시간,기온
0,2013-03-01 01:00,10.6
1,2013-03-01 02:00,10.5
2,2013-03-01 03:00,11.0
3,2013-03-01 04:00,10.3
4,2013-03-01 05:00,9.6
5,2013-03-01 06:00,9.0
6,2013-03-01 07:00,5.9
7,2013-03-01 08:00,5.2
8,2013-03-01 09:00,3.6
9,2013-03-01 10:00,3.7


In [4]:
test_list = [0,1,2,3,4,5]
test_list.insert(3,10)
test_list

[0, 1, 2, 10, 3, 4, 5]

In [6]:
for a, one in enumerate(test_list):
    print(type(a))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
